### Importación de librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from NSGAII import *

### Generar una poblaciónn inicial

In [2]:
def generar_poblacion(n_soluciones, n_variables, limite_inferior=0, limite_superior=np.inf, tipo_codificacion="continua"):
    """
    Genera una población de soluciones aleatorias de forma genérica.

    Args:
        n_soluciones (int): El número de individuos a generar (ej. 10000).
        n_variables (int): El número de variables de decisión para cada individuo.
        limite_inferior (float or int): El límite inferior del rango de valores.
        limite_superior (float or int): El límite superior del rango de valores.
        tipo_codificacion (str): "continua" para valores flotantes (por defecto) o
                                 "discreta" para valores enteros.

    Returns:
        np.array: una matriz de tamaño (n_soluciones, n_variables) con valores
                  aleatorios dentro del rango y tipo especificados.
    """
    if tipo_codificacion == "continua":
        # Genera valores flotantes en [0, 1) y luego los escala al rango deseado.
        # Fórmula: min + (max - min) * rand()
        poblacion = limite_inferior + (limite_superior - limite_inferior) * np.random.rand(n_soluciones, n_variables)
    
    elif tipo_codificacion == "discreta":
        # Genera valores enteros directamente en el rango [limite_inferior, limite_superior].
        # np.random.randint tiene un límite superior exclusivo, por eso se le suma 1.
        poblacion = np.random.randint(low=limite_inferior, high=limite_superior + 1, size=(n_soluciones, n_variables))
        
    else:
        raise ValueError("El 'tipo_codificacion' debe ser 'continua' o 'discreta'.")
        
    return poblacion

### DTLZ1

In [ ]:
def dtlz1(x, n_objetivos):
    """
    Calcula los valores de los M objetivos para una solución 'x' del problema DTLZ1.

    Args:
        x (np.array): Un vector de decisión de tamaño (n_variables,).
        n_objetivos (int): El número de objetivos (M).

    Returns:
        np.array: Un vector con los M valores de los objetivos calculados.
    """
    # El número de variables (n) y objetivos (M) están relacionados.
    # Usualmente, n = M + k - 1. Un valor estándar para k es 5.
    # Por lo tanto, n_variables debe ser >= n_objetivos.
    if x.shape[0] < n_objetivos:
        raise ValueError("El número de variables debe ser al menos el número de objetivos.")

    # 1. Separar las variables en x_I y x_II
    # x_I tiene las primeras M-1 variables
    # x_II tiene el resto
    m = n_objetivos
    x_I = x[:m-1]
    x_II = x[m-1:]

    # 2. Calcular la función g(x_II)
    # Esta es la "función trampa" que debe ser minimizada a cero
    termino_suma = np.sum((x_II - 0.5)**2 - np.cos(20 * np.pi * (x_II - 0.5)))
    g = 100 * (len(x_II) + termino_suma)

    # 3. Calcular los M objetivos f(x)
    objetivos = np.zeros(m)
    factor_g = 0.5 * (1 + g)

    # El cálculo se puede hacer de forma iterativa
    cos_prod = 1.0
    for i in range(m - 1):
        # f_i = factor_g * prod(cos) * sin
        objetivos[i] = factor_g * cos_prod * np.sin(x[i] * np.pi / 2)
        cos_prod *= np.cos(x[i] * np.pi / 2)

    # El primer objetivo es un caso especial (solo cosenos)
    objetivos[m-1] = factor_g * cos_prod
    
    # La fórmula matemática suele listar de f1 a fM.
    # La implementación es más fácil si la recorremos al revés,
    # así que invertimos el resultado para que coincida con la definición.
    return objetivos[::-1]

### DTLZ2

In [4]:
def dtlz2(x, n_objetivos):
    """
    Calcula los valores de los M objetivos para una solución 'x' del problema DTLZ2.

    Args:
        x (np.array): Un vector de decisión de tamaño (n_variables,).
        n_objetivos (int): El número de objetivos (M).

    Returns:
        np.array: Un vector con los M valores de los objetivos calculados.
    """
    if x.shape[0] < n_objetivos:
        raise ValueError("El número de variables debe ser al menos el número de objetivos.")

    m = n_objetivos
    # Para DTLZ2, se recomienda k=10. n_variables = n_objetivos + k - 1
    
    # 1. Separar las variables
    x_I = x[:m-1]
    x_II = x[m-1:]

    # 2. Calcular la función g(x_II)
    # Es mucho más simple que en DTLZ1, sin trampas
    g = np.sum((x_II - 0.5)**2)

    # 3. Calcular los M objetivos f(x)
    objetivos = np.zeros(m)
    factor_g = 1 + g

    # El cálculo es idéntico a DTLZ1, solo cambia el factor_g
    cos_prod = 1.0
    for i in range(m - 1):
        objetivos[i] = factor_g * cos_prod * np.sin(x[i] * np.pi / 2)
        cos_prod *= np.cos(x[i] * np.pi / 2)
    
    objetivos[m-1] = factor_g * cos_prod
    
    return objetivos[::-1]

In [11]:
def dtlz7(x, n_objetivos):
    """
    Calcula los valores de los M objetivos para una solución 'x' del problema DTLZ7.

    Args:
        x (np.array): Un vector de decisión de tamaño (n_variables,).
        n_objetivos (int): El número de objetivos (M).

    Returns:
        np.array: Un vector con los M valores de los objetivos calculados.
    """
    if x.shape[0] < n_objetivos:
        raise ValueError("El número de variables debe ser al menos el número de objetivos.")

    m = n_objetivos
    # Para DTLZ7, se recomienda k=20. n_variables = n_objetivos + k - 1

    objetivos = np.zeros(m)
    
    # 1. Calcular los primeros M-1 objetivos
    objetivos[:m-1] = x[:m-1]

    # 2. Separar x_II y calcular g(x_II)
    x_II = x[m-1:]
    g = 1 + (9.0 / len(x_II)) * np.sum(x_II)

    # 3. Calcular h
    f_primeros = objetivos[:m-1]
    termino_suma_h = np.sum( (f_primeros / (1.0 + g)) * (1 + np.sin(3.0 * np.pi * f_primeros)) )
    h = m - termino_suma_h
    
    # 4. Calcular el último objetivo, f_M
    objetivos[m-1] = (1 + g) * h

    return objetivos

### Obtener el fitness de una población inicial

In [ ]:
def evaluar_poblacion(poblacion, funcion_evaluacion, n_objetivos):
    """
    Evalúa cada individuo de una población usando una función de fitness específica.

    Args:
        poblacion (np.array): Matriz (n_soluciones, n_variables) que contiene la población.
        funcion_evaluacion (function): La función a usar para evaluar cada individuo
                                       (ej. evaluar_dtlz1, evaluar_dtlz2).
        n_objetivos (int): El número de objetivos a calcular, requerido por la
                           función de evaluación.

    Returns:
        list: Una lista donde cada elemento es una sublista con la forma:
              [cromosoma_individuo (np.array), fitness_calculado (np.array)].
    """
    poblacion_evaluada = []
    for individuo in poblacion:
        fitness = funcion_evaluacion(individuo, n_objetivos)
        
        poblacion_evaluada.append([individuo, fitness])
        
    return poblacion_evaluada

### Generando una población inicial

In [7]:
poblacion_aleatoria = generar_poblacion(100,3,0.0, 1.0, 'continua')
print(f"La cantidad de individuos generados fueron: {len(poblacion_aleatoria)}")

La cantidad de individuos generados fueron: 100


### Probando DTZ1

In [ ]:
num_objetivos = 2


### Probando las funciones

In [14]:
N_SOLUCIONES = 10000
N_OBJETIVOS =  2# Probemos con 3 objetivos

# --- DTLZ1 ---
k1 = 5
n_vars_1 = N_OBJETIVOS + k1 - 1
pob1 = generar_poblacion(N_SOLUCIONES, n_vars_1, 0, 1, 'continua')
sol_eval_1 = dtlz1(pob1[0], N_OBJETIVOS)
print(f"DTLZ1 (3 obj, {n_vars_1} vars) - Ejemplo evaluación: {sol_eval_1}")

# --- DTLZ2 ---
k2 = 10
n_vars_2 = N_OBJETIVOS + k2 - 1
pob2 = generar_poblacion(N_SOLUCIONES, n_vars_2, 0, 1, 'continua')
sol_eval_2 = dtlz2(pob2[0], N_OBJETIVOS)
print(f"DTLZ2 (3 obj, {n_vars_2} vars) - Ejemplo evaluación: {sol_eval_2}")

# --- DTLZ7 ---
k7 = 20
n_vars_7 = N_OBJETIVOS + k7 - 1
pob7 = generar_poblacion(N_SOLUCIONES, n_vars_7, 0, 1, 'continua')
sol_eval_7 = dtlz7(pob7[0], N_OBJETIVOS)
print(f"DTLZ7 (3 obj, {n_vars_7} vars) - Ejemplo evaluación: {sol_eval_7}")

DTLZ1 (3 obj, 6 vars) - Ejemplo evaluación: [238.05491012  46.10132652]
DTLZ2 (3 obj, 11 vars) - Ejemplo evaluación: [1.2930224  0.99135965]
DTLZ7 (3 obj, 21 vars) - Ejemplo evaluación: [ 0.73940157 13.57765358]
